In [1]:
import os, sys, inspect
# sys.path.append('../..')

import pandas as pd
from pathlib import Path
import glob as glob
import os
import numpy as np
import re
import json
from datetime import datetime, timedelta, date
from connection import SCORE_DB_CONN, ANALYTICS_DB_CONN
from avay_bq import AvayBQServiceAcc
from da_utils.client.bigquery_client import BigQueryClient
from da_utils.client.google_sheets_client import GoogleSheetsClient
from da_utils.repository.google_sheets.google_sheets_repository import GoogleSheetsRepository
from da_utils.repository.bigquery.bigquery_repository import BigqueryRepository



pd.set_option("display.max_row", 100)

In [2]:
adc_path = '/home/linhnguyen/.config/application_default_credentials.json'
avay_bq_acc = AvayBQServiceAcc()
bq_client = BigQueryClient(billing_project_id='prj-ts-p-analytic-8057', adc_file_path_str=adc_path)
bq_client_avay = BigQueryClient(billing_project_id='avay-a9925', adc_file_path_str=adc_path)
bq_repo = BigqueryRepository(bq_client)
bq_repo_avay = BigqueryRepository(bq_client_avay)

/home/linhnguyen/.venv/lib/python3.9/site-packages/google/auth/_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [3]:
Qlead = bq_repo_avay.get_data_from_query_into_pandas(
        f"""

with rangedate as (
    select
date('2022-12-01') as sdate,
date('2023-05-31') as edate)
,qualified as(
    select 
        lead_id
        ,loan_date as lead_sent_date
        ,case  when lower(lead_source) like '%acebook%' then 'facebook_ads'
        when lower(lead_source) like 'google%' then 'google_ads'
        when lower(lead_source) in ('accesstrade', 'accesstrade cps','vaycucde') then 'accesstrade'
        when lower(lead_source) in ('masoffer','vaysieude') then 'masoffer'
        when lower(lead_source) = 'dircrezu' then 'dircrezu'
        when lower(lead_source) like 'viettel%' then 'VTP'
        when lower(lead_source) in ('mgid','adfly') then 'Mgid'
        else 'Others' end as lead_source
        ,bank_name
        ,CASE
        WHEN TELCO_SCORE >= 650 THEN '[650+]'
        WHEN TELCO_SCORE >= 630 THEN '[630 - 649]'
        WHEN TELCO_SCORE >= 600 THEN '[600 - 629]'
        WHEN SCORE_ID IS NULL THEN 'NON-VT'
        END AS SCORE_RANGE
        ,CASE  WHEN desired_amount >= 25000000 THEN '3.[25M+)'
        WHEN desired_amount >= 15000000 THEN '2.[15 - 25M)'
        ELSE '1.[0-15M)'
        END AS desired_amount_range
    from `avay-a9925.datamart.avay_compound`, rangedate
    where loan_date between sdate and edate and loan_status='accepted'
    order by 1,2,3,4
)
select * from qualified
where bank_name = 'Lotte Finance'
        """
)
Qlead

,lead_id,lead_sent_date,lead_source,bank_name,SCORE_RANGE,desired_amount_range
0,13338105,2022-12-01,google_ads,Lotte Finance,[650+],1.[0-15M)
1,13338228,2022-12-01,accesstrade,Lotte Finance,[650+],1.[0-15M)
2,13338236,2022-12-01,VTP,Lotte Finance,[600 - 629],3.[25M+)
3,13338253,2022-12-01,accesstrade,Lotte Finance,[650+],1.[0-15M)
4,13338273,2022-12-01,VTP,Lotte Finance,[600 - 629],1.[0-15M)
...,...,...,...,...,...,...
40586,14100896,2023-05-23,VTP,Lotte Finance,[600 - 629],1.[0-15M)
40587,14100932,2023-05-23,VTP,Lotte Finance,[630 - 649],2.[15 - 25M)
40588,14100980,2023-05-23,VTP,Lotte Finance,[630 - 649],3.[25M+)
40589,14101041,2023-05-23,google_ads,Lotte Finance,NON-VT,1.[0-15M)


In [4]:
Disbursal1 = bq_repo_avay.get_data_from_query_into_pandas(
        f"""

with rangedate as (
    select
date('2022-12-01') as sdate,
date('2023-05-31') as edate
)
,Disbursal as(
    select 
  lead_id,
  case when disbursed_date ='2000-01-01' then loan_date else disbursed_date end as disbursed_date,
  cast(loan_amount as int64) as loan_amount,
  cast(commission as int64) as commission
from `avay-a9925.datamart.avay_compound`, rangedate
    where loan_date between sdate and edate and disbursed_date is not null 
    order by 1,2
)

select * from Disbursal
        """
)
Disbursal1

,lead_id,disbursed_date,loan_amount,commission
0,13338104,2022-12-02,10000000,600000
1,13338105,2022-12-01,<NA>,168000
2,13338212,2022-12-15,25000000,1375000
3,13338236,2022-12-01,<NA>,168000
4,13338265,2022-12-10,0,620000
...,...,...,...,...
14373,13897185,2023-03-31,<NA>,168000
14374,13897199,2023-03-31,<NA>,168000
14375,13897269,2023-03-31,<NA>,168000
14376,13897337,2023-03-31,<NA>,168000


In [5]:
Disbursal2 = bq_repo.get_data_from_query_into_pandas(
        f"""

with rangedate as (
    select
date('2022-12-01') as sdate,
date('2023-05-31') as edate
)
,Disbursal as(
    select 
        lead_id
        ,case when disbursed_date ='2000-01-01' then batch_date else disbursed_date end as disbursed_date
        ,loan_amount
        ,commission
    from `prj-ts-p-analytic-8057.da.disbursals`, rangedate
    where batch_date between sdate and edate and channel = 'avay'
    order by 1,2
)

select * from Disbursal
        """
)
Disbursal2

,lead_id,disbursed_date,loan_amount,commission
0,0,2022-12-02,0,450000.0
1,0,2022-12-03,0,450000.0
2,0,2022-12-03,0,450000.0
3,0,2022-12-03,0,450000.0
4,0,2022-12-03,0,450000.0
...,...,...,...,...
15937,14079301,2023-05-20,8306000,415300.0
15938,14083174,2023-05-20,30921000,1546050.0
15939,14085094,2023-05-20,21500000,1075000.0
15940,14086589,2023-05-20,21500000,1075000.0


In [6]:
Disbursal = Disbursal2.combine_first(Disbursal1)
Disbursal = Disbursal.drop_duplicates(subset=['lead_id'])


In [7]:
leads_with_disbursement = Qlead.merge(Disbursal, how = 'left').assign(
    lead_sent_month=lambda x: x.lead_sent_date.astype(str).str[0:7],
    loan_sent_month=lambda x: x.disbursed_date.astype(str).str[0:7])
leads_with_disbursement["loan"] = np.where((leads_with_disbursement["disbursed_date"].notnull())&(leads_with_disbursement["loan_sent_month"] == leads_with_disbursement["lead_sent_month"]) ,1,0)
leads_with_disbursement["loan_next_month"] = np.where((leads_with_disbursement["disbursed_date"].notnull())&(leads_with_disbursement["loan_sent_month"] != leads_with_disbursement["lead_sent_month"]) ,1,0)

leads_with_disbursement[(leads_with_disbursement['bank_name']=='FE Credit')&(leads_with_disbursement['lead_sent_date']>=date(2023,3,1))&
                        (leads_with_disbursement["disbursed_date"].notnull())]

,lead_id,lead_sent_date,lead_source,bank_name,SCORE_RANGE,desired_amount_range,disbursed_date,loan_amount,commission,lead_sent_month,loan_sent_month,loan,loan_next_month


In [8]:
leads_with_disbursement_pivot_month = leads_with_disbursement.groupby(["bank_name", "SCORE_RANGE", "desired_amount_range", "lead_sent_month"]).agg({"lead_id":"count", "loan":"sum", "commission":"mean", "loan_next_month":"sum"}).reset_index()
leads_with_disbursement_pivot_month['CR_in_month'] = leads_with_disbursement_pivot_month['loan']/leads_with_disbursement_pivot_month['lead_id']
leads_with_disbursement_pivot_month['CR_next_month'] = leads_with_disbursement_pivot_month['loan_next_month']/leads_with_disbursement_pivot_month['lead_id']
leads_with_disbursement_pivot_month['CR'] = (leads_with_disbursement_pivot_month['loan_next_month']+ leads_with_disbursement_pivot_month['loan'])/leads_with_disbursement_pivot_month['lead_id']
leads_with_disbursement_pivot_month[(leads_with_disbursement_pivot_month['bank_name']=='FE Credit')&(leads_with_disbursement_pivot_month['lead_sent_month']=='2023-03')]

,bank_name,SCORE_RANGE,desired_amount_range,lead_sent_month,lead_id,loan,commission,loan_next_month,CR_in_month,CR_next_month,CR


In [9]:
client = GoogleSheetsClient()
repo = GoogleSheetsRepository(client)
workbook = repo.open_spreadsheet_by_key("1OrcjB40OTk_WTUTGyWwvck9-iKMesfF2jWBAHL5uw00")

repo.write_df_to_sheet(workbook.worksheet("data"),leads_with_disbursement_pivot_month)